In [1]:
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [18]:
import pandas as pd
import numpy as np
import torch
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
    classification_report
)

In [96]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

In [97]:
'''
# customer_country, customer_country.1 날리고 response_orporate만 남기기
'''
drop_col = ['id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver', 'product_subcategory', 'product_modelname', 'ver_win_ratio_per_bu', 'business_area', 'business_subarea', 'ver_win_rate_x', 'com_reg_ver_win_rate', 'historical_existing_cnt']
df_train.drop(drop_col, axis=1, inplace=True)
df_test.drop(drop_col, axis=1, inplace=True)

In [98]:
label_columns = list(df_train.columns)

label_columns.remove("is_converted")

In [99]:
'''
# 수치형 변수 전체 Standardizaiton, Normalization 실행
'''
# Initialize scalers
scaler_standard = StandardScaler()
scaler_minmax = MinMaxScaler()
columns_to_ST = ["lead_desc_length"]

# Apply Standardization to each column
for column in columns_to_ST:
    # Standardization
    df_train[column + '_standardized'] = scaler_standard.fit_transform(df_train[[column]])
    df_test[column + '_standardized'] = scaler_standard.fit_transform(df_test[[column]])
    # Drop the original column
    df_train.drop(columns=[column], inplace=True)
    df_test.drop(columns=[column], inplace=True)

In [100]:
'''
# expected_timeline 에서 특정 단어 필터링
'''
# 새로운 열을 추가하기 위한 함수 정의
def contains_budget(value):
    if 'budget' in str(value).lower():  # 대소문자 구분없이 'budget'이 포함되어 있는지 확인
        return 1
    else:
        return 0

# "expected_timeline" 열에서 함수를 적용하여 새로운 열 생성
df_train['contains_budget'] = df_train['expected_timeline'].apply(contains_budget)
df_test['contains_budget'] = df_test['expected_timeline'].apply(contains_budget)

# 새로운 열을 추가하기 위한 함수 정의
def contains_etc(value):
    if 'etc' in str(value).lower():  # 대소문자 구분없이 'budget'이 포함되어 있는지 확인
        return 1
    else:
        return 0

# "expected_timeline" 열에서 함수를 적용하여 새로운 열 생성
df_train['contains_etc'] = df_train['expected_timeline'].apply(contains_etc)
df_test['contains_etc'] = df_test['expected_timeline'].apply(contains_etc)

# 새로운 열을 추가하기 위한 함수 정의
def contains_hence(value):
    if 'hence' in str(value).lower():  # 대소문자 구분없이 'budget'이 포함되어 있는지 확인
        return 1
    else:
        return 0

# "expected_timeline" 열에서 함수를 적용하여 새로운 열 생성
df_train['contains_hence'] = df_train['expected_timeline'].apply(contains_hence)
df_test['contains_hence'] = df_test['expected_timeline'].apply(contains_hence)

# 새로운 열을 추가하기 위한 함수 정의
def contains_although(value):
    if 'although' in str(value).lower():  # 대소문자 구분없이 'budget'이 포함되어 있는지 확인
        return 1
    else:
        return 0

# "expected_timeline" 열에서 함수를 적용하여 새로운 열 생성
df_train['contains_although'] = df_train['expected_timeline'].apply(contains_although)
df_test['contains_although'] = df_test['expected_timeline'].apply(contains_although)

# 새로운 열을 추가하기 위한 함수 정의
def contains_more(value):
    if 'more' in str(value).lower():  # 대소문자 구분없이 'budget'이 포함되어 있는지 확인
        return 1
    else:
        return 0

# "expected_timeline" 열에서 함수를 적용하여 새로운 열 생성
df_train['contains_more'] = df_train['expected_timeline'].apply(contains_more)
df_test['contains_more'] = df_test['expected_timeline'].apply(contains_more)

# 새로운 열을 추가하기 위한 함수 정의
def contains_year(value):
    if 'year' in str(value).lower():  # 대소문자 구분없이 'budget'이 포함되어 있는지 확인
        return 1
    else:
        return 0

# "expected_timeline" 열에서 함수를 적용하여 새로운 열 생성
df_train['contains_year'] = df_train['expected_timeline'].apply(contains_year)
df_test['contains_year'] = df_test['expected_timeline'].apply(contains_year)

In [101]:
# Calculate the most frequent value (mode) for each column
modes = df_train.mode().iloc[0]

# Fill missing (NA) values with the mode
df_train.fillna(modes, inplace=True)

# Calculate the most frequent value (mode) for each column
modes = df_test.mode().iloc[0]

# Fill missing (NA) values with the mode
df_test.fillna(modes, inplace=True)

In [102]:
'''
df_train과 df_test에서 숫자 형식인거 다 str로 바꾸기 (일부제외)
'''

# 변환할 라벨들의 리스트
labels_to_convert = [
    "bant_submit",
    "lead_desc_length_standardized",
    "ver_cus",
    "ver_pro"
]

# df_test와 df_train에서 라벨들의 데이터 타입을 str로 변환
for label in labels_to_convert:
    if label in df_test.columns:
        df_test[label] = df_test[label].astype(str)
    if label in df_train.columns:
        df_train[label] = df_train[label].astype(str)

df_train['is_converted'] = df_train['is_converted'].astype(int)
df_test['is_converted'] = df_test['is_converted'].astype(int)

In [103]:
x_train, x_val, y_train, y_val = train_test_split(
    df_train.drop("is_converted", axis=1),
    df_train["is_converted"],
    test_size=0.2,
    shuffle=True,
    random_state=400,
)

In [104]:
cat_features_col = list(x_train.columns)

In [105]:
catboost_model = CatBoostClassifier(
    verbose=0, 
    iterations=1000, 
    depth=6, 
    learning_rate=0.1, 
    loss_function='CrossEntropy',
    one_hot_max_size=5,
    cat_features=cat_features_col
    )

In [106]:
catboost_model.fit(x_train, y_train)

In [107]:
feature_importances = catboost_model.get_feature_importance()

# 특성 중요도 출력
for score, name in sorted(zip(feature_importances, x_train.columns), reverse=True):
    print(f"{name}: {score}")

lead_owner: 26.61783388790074
customer_idx: 19.100717201963278
inquiry_type: 8.099742493671643
lead_desc_length_standardized: 6.755515978308485
product_category: 6.580650087574272
response_corporate: 6.277030512051421
customer_type: 5.4715662944799295
customer_job: 4.4775931617496125
customer_country.1: 3.828743922292745
customer_position: 3.588789070158321
business_unit: 1.7841640603617246
expected_timeline: 1.777658703743146
bant_submit: 1.697327145439356
ver_cus: 1.6877583240879317
enterprise: 0.750554974770155
customer_country: 0.695513235501489
ver_pro: 0.6490025188554939
contains_etc: 0.0807046022794914
contains_more: 0.042138141247677265
contains_year: 0.036436532137434995
contains_hence: 0.0005591514256589993
contains_budget: 0.0
contains_although: 0.0


In [108]:
# 모델을 사용하여 검증 데이터셋에 대한 예측 수행
y_val_pred = catboost_model.predict(x_val)
print (y_val_pred)

y_val_pred = np.array([True if pred == 'True' else False for pred in y_val_pred])

print("Classification Report:\n", classification_report(y_val, y_val_pred))
print("오차행렬:\n", confusion_matrix(y_val, y_val_pred, labels=[True, False]))
print("\n정확도: {:.4f}".format(accuracy_score(y_val, y_val_pred)))
print("정밀도: {:.4f}".format(precision_score(y_val, y_val_pred, labels=[True, False])))
print("재현율: {:.4f}".format(recall_score(y_val, y_val_pred)))
print("F1: {:.4f}".format(f1_score(y_val, y_val_pred, labels=[True, False])))

[0 0 0 ... 0 0 0]
Classification Report:
               precision    recall  f1-score   support

           0       0.92      1.00      0.96     10913
           1       0.00      0.00      0.00       947

    accuracy                           0.92     11860
   macro avg       0.46      0.50      0.48     11860
weighted avg       0.85      0.92      0.88     11860

오차행렬:
 [[    0   947]
 [    0 10913]]

정확도: 0.9202
정밀도: 0.0000
재현율: 0.0000
F1: 0.0000


/home/elicer/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/elicer/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/elicer/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/elicer/.local/lib/python3.10/si

In [109]:
x_test = df_test.drop(["is_converted", "id"], axis=1)

test_pred = catboost_model.predict(x_test)

df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = test_pred
df_sub['is_converted'] = df_sub['is_converted'].astype(bool)
print(df_sub['is_converted'])

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

0        True
1        True
2        True
3       False
4       False
        ...  
5266    False
5267     True
5268    False
5269    False
5270     True
Name: is_converted, Length: 5271, dtype: bool
